In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import fsolve

class MertonCreditModel:
    def __init__(self, equity_val, equity_vol, debt, r, T):
        """
        :param equity_val: Market Capitalization
        :param equity_vol: Annualized Volatility of stock returns
        :param debt: Book value of total liabilities (face value of debt)
        :param r: Risk-free rate (e.g., 10-year Treasury yield)
        :param T: Time to maturity (usually 1 year for PD)
        """
        self.E = equity_val
        self.sigma_E = equity_vol
        self.D = debt
        self.r = r
        self.T = T

    def _equations(self, variables):
        Va, sigma_a = variables
        
        d1 = (np.log(Va / self.D) + (self.r + 0.5 * sigma_a**2) * self.T) / (sigma_a * np.sqrt(self.T))
        d2 = d1 - sigma_a * np.sqrt(self.T)
        
        # Equation 1: Black-Scholes for Equity
        eq1 = Va * norm.cdf(d1) - self.D * np.exp(-self.r * self.T) * norm.cdf(d2) - self.E
        
        # Equation 2: Relation between Equity Vol and Asset Vol (Option Delta)
        # sigma_E = (Va / E) * Delta * sigma_a
        delta = norm.cdf(d1)
        eq2 = (Va / self.E) * delta * sigma_a - self.sigma_E
        
        return [eq1, eq2]

    def calibrate(self):
        # Initial guesses: Va approx E + D, sigma_a approx sigma_E * (E/(E+D))
        init_guess = [self.E + self.D, self.sigma_E * self.E / (self.E + self.D)]
        Va, sigma_a = fsolve(self._equations, init_guess)
        
        d1 = (np.log(Va / self.D) + (self.r + 0.5 * sigma_a**2) * self.T) / (sigma_a * np.sqrt(self.T))
        d2 = d1 - sigma_a * np.sqrt(self.T)
        
        # Probability of Default (Real-world measure would use drift 'mu' instead of 'r')
        # We use the Risk-Neutral Probability of Default
        pd = norm.cdf(-d2)
        
        # Distance to Default (DD)
        dd = (np.log(Va / self.D) + (self.r - 0.5 * sigma_a**2) * self.T) / (sigma_a * np.sqrt(self.T))
        
        return {
            "Asset Value": Va,
            "Asset Volatility": sigma_a,
            "Distance to Default": dd,
            "Prob of Default (PD)": pd
        }

# --- Example Usage (Hypothetical Tech Company) ---
# Market Cap: $100B, Equity Vol: 40%, Total Debt: $60B, Rate: 4%, Time: 1yr
model = MertonCreditModel(equity_val=100e9, equity_vol=0.40, debt=60e9, r=0.04, T=1.0)
results = model.calibrate()

print("Merton Model Results: ")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

Merton Model Results: 
Asset Value: 157647165914.7526
Asset Volatility: 0.2537
Distance to Default: 3.8379
Prob of Default (PD): 0.0001
